In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Chronic_obstructive_pulmonary_disease_(COPD)"
cohort = "GSE64593"

# Input paths
in_trait_dir = "../../input/GEO/Chronic_obstructive_pulmonary_disease_(COPD)"
in_cohort_dir = "../../input/GEO/Chronic_obstructive_pulmonary_disease_(COPD)/GSE64593"

# Output paths
out_data_file = "../../output/preprocess/Chronic_obstructive_pulmonary_disease_(COPD)/GSE64593.csv"
out_gene_data_file = "../../output/preprocess/Chronic_obstructive_pulmonary_disease_(COPD)/gene_data/GSE64593.csv"
out_clinical_data_file = "../../output/preprocess/Chronic_obstructive_pulmonary_disease_(COPD)/clinical_data/GSE64593.csv"
json_path = "../../output/preprocess/Chronic_obstructive_pulmonary_disease_(COPD)/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"The Role of Interleukin-23 in the Early Development of Emphysema in HIV1+ Smokers [Affymetrix]"
!Series_summary	"Matrix metalloproteinase-9 (MMP-9) expression is up-regulated in alveolar macrophages (AM) of HIV1+ smokers who develop emphysema. Based on the knowledge that lung epithelial lining fluid (ELF) of HIV1+ smokers has increased levels of inflammatory cytokines compared to HIV1- smokers, we hypothesized up-regulation of lung cytokines in HIV1+ smokers may be functionally related to increased MMP-9 expression. Cytokine arrays evaluated cytokine protein levels in ELF obtained from 5 groups of individuals: HIV1‾ healthy nonsmokers, HIV1‾ healthy smokers, HIV1‾ smokers with low diffusing capacity (DLCO) , HIV1 + nonsmokers, and HIV1 + smokers with low DLCO. Among several pro-inflammatory cytokines elevated in ELF associated with smoking and HIV1+, increased levels of the Th17-related cytokine IL-23 were found in HIV1- smokers with low DLCO and 

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Analysis
# Based on the background information, this dataset appears to be analyzing gene expression
# specifically for MMP-9 in alveolar macrophages using Affymetrix arrays.
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# For COPD/emphysema trait:
# Looking at the sample characteristics, there's no direct mention of COPD, emphysema, or DLCO status
# The study examines HIV+ smokers with low DLCO as having emphysema, but we don't have this info in the characteristics
# We cannot reliably infer COPD status from just HIV status without DLCO measurements
trait_row = None  # No direct or reliably inferable COPD trait data

# Age data - Not available in the sample characteristics
age_row = None

# Gender data - Not available in the sample characteristics
gender_row = None

# Conversion functions defined for completeness
def convert_trait(value):
    return None  # Not applicable since we don't have COPD trait data

def convert_age(value):
    return None  # Not used but included for completeness

def convert_gender(value):
    return None  # Not used but included for completeness

# 3. Save Metadata
# Determine trait data availability
is_trait_available = trait_row is not None  # Will be False

# Initial filtering validation
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Skip this step as trait_row is None (no reliable COPD trait data available)
print("No COPD/emphysema trait data available for extraction.")


No COPD/emphysema trait data available for extraction.


### Step 3: Gene Data Extraction

In [4]:
# 1. Get the SOFT and matrix file paths again 
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"Matrix file found: {matrix_file}")

# 2. Use the get_genetic_data function from the library to get the gene_data
try:
    gene_data = get_genetic_data(matrix_file)
    print(f"Gene data shape: {gene_data.shape}")
    
    # 3. Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")


Matrix file found: ../../input/GEO/Chronic_obstructive_pulmonary_disease_(COPD)/GSE64593/GSE64593_series_matrix.txt.gz
Gene data shape: (54675, 34)
First 20 gene/probe identifiers:
Index(['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at', '1294_at',
       '1316_at', '1320_at', '1405_i_at', '1431_at', '1438_at', '1487_at',
       '1494_f_at', '1552256_a_at', '1552257_a_at', '1552258_at', '1552261_at',
       '1552263_at', '1552264_a_at', '1552266_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# These identifiers like '1007_s_at', '1053_at', etc. are Affymetrix probe IDs
# They are not human gene symbols and need to be mapped to gene symbols

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Analyze the gene annotation dataframe to identify which columns contain the gene identifiers and gene symbols
print("\nGene annotation preview:")
print(f"Columns in gene annotation: {gene_annotation.columns.tolist()}")
print(preview_df(gene_annotation, n=5))

# Let's look for platform information in the SOFT file to understand the annotation better
print("\nSearching for platform information in SOFT file:")
with gzip.open(soft_file, 'rt') as f:
    for i, line in enumerate(f):
        if '!Series_platform_id' in line:
            print(line.strip())
            break
        if i > 100:  # Limit search to first 100 lines
            print("Platform ID not found in first 100 lines")
            break

# Check if the SOFT file includes any reference to gene symbols
print("\nSearching for gene symbol information in SOFT file:")
with gzip.open(soft_file, 'rt') as f:
    gene_symbol_lines = []
    for i, line in enumerate(f):
        if 'GENE_SYMBOL' in line or 'gene_symbol' in line.lower() or 'symbol' in line.lower():
            gene_symbol_lines.append(line.strip())
        if i > 1000 and len(gene_symbol_lines) > 0:  # Limit search but ensure we found something
            break
    
    if gene_symbol_lines:
        print("Found references to gene symbols:")
        for line in gene_symbol_lines[:5]:  # Show just first 5 matches
            print(line)
    else:
        print("No explicit gene symbol references found in first 1000 lines")

# Look for alternative annotation files or references in the directory
print("\nChecking for additional annotation files in the directory:")
all_files = os.listdir(in_cohort_dir)
print([f for f in all_files if 'annotation' in f.lower() or 'platform' in f.lower() or 'gpl' in f.lower()])



Gene annotation preview:
Columns in gene annotation: ['ID', 'GB_ACC', 'SPOT_ID', 'Species Scientific Name', 'Annotation Date', 'Sequence Type', 'Sequence Source', 'Target Description', 'Representative Public ID', 'Gene Title', 'Gene Symbol', 'ENTREZ_GENE_ID', 'RefSeq Transcript ID', 'Gene Ontology Biological Process', 'Gene Ontology Cellular Component', 'Gene Ontology Molecular Function']
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database', 'GenBank', 

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify which columns contain the gene identifiers and gene symbols
# The column 'ID' in gene_annotation contains identifiers that match the gene expression data index
# The column 'Gene Symbol' contains the gene symbols we need to map to
prob_col = 'ID'
gene_col = 'Gene Symbol'

# 2. Extract the mapping dataframe using the get_gene_mapping function
gene_mapping = get_gene_mapping(gene_annotation, prob_col, gene_col)
print(f"Gene mapping dataframe shape: {gene_mapping.shape}")
print("Sample of gene mapping:")
print(gene_mapping.head())

# 3. Apply the gene mapping to convert probe-level measurements to gene expression data
gene_data = apply_gene_mapping(gene_data, gene_mapping)
print(f"Gene expression data shape after mapping: {gene_data.shape}")
print("Sample of gene expression data:")
print(gene_data.head())

# Check how many unique genes we have after mapping
print(f"Number of unique genes after mapping: {len(gene_data.index)}")


Gene mapping dataframe shape: (45782, 2)
Sample of gene mapping:
          ID              Gene
0  1007_s_at  DDR1 /// MIR4640
1    1053_at              RFC2
2     117_at             HSPA6
3     121_at              PAX8
4  1255_g_at            GUCA1A


Gene expression data shape after mapping: (21278, 34)
Sample of gene expression data:
          GSM1575085  GSM1575086  GSM1575087  GSM1575088  GSM1575089  \
Gene                                                                   
A1BG        92.13980     87.1394    136.8800   169.79900    149.5720   
A1BG-AS1    79.08870     29.9709     26.5951    97.24380     10.3971   
A1CF        65.27392    146.8920    157.2644   117.82857     98.7029   
A2M       5640.59000  10535.9921   5956.3960  4601.38610  10920.6929   
A2M-AS1     95.51210     66.9938     72.5051    78.37100    102.9160   

          GSM1575090  GSM1575091  GSM1575092   GSM1575093  GSM1575094  ...  \
Gene                                                                   ...   
A1BG        137.5130    110.3330   180.97400    198.45200    292.8590  ...   
A1BG-AS1     86.5203    100.2560    14.18130     20.68470     98.5018  ...   
A1CF         93.7718    174.4951   122.40554     38.26969    186.8891  ...   
A2M        2494.527

### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
try:
    normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
    print(f"Gene data shape after normalization: {normalized_gene_data.shape}")
    
    # Save normalized gene data
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    normalized_gene_data.to_csv(out_gene_data_file)
    print(f"Gene data saved to {out_gene_data_file}")
except Exception as e:
    print(f"Error during gene symbol normalization: {e}")
    normalized_gene_data = gene_data
    print("Using original gene data (without normalization).")

# 2. Since we identified in step 2 that there's no COPD trait information available,
# we cannot create clinical features or link the data
print("\nNo COPD trait data available for this cohort.")

# 3. Conduct initial validation since we can't do final validation without trait data
is_usable = validate_and_save_cohort_info(
    is_final=False,  # Use initial validation when trait data is unavailable
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=False  # We determined there's no COPD trait data in step 2
)

print(f"Dataset usability status: {'Usable' if is_usable else 'Not usable'}")
print("Dataset deemed not usable for COPD analysis due to missing trait data - linked data not created")

Gene data shape after normalization: (19845, 34)


Gene data saved to ../../output/preprocess/Chronic_obstructive_pulmonary_disease_(COPD)/gene_data/GSE64593.csv

No COPD trait data available for this cohort.
Dataset usability status: Not usable
Dataset deemed not usable for COPD analysis due to missing trait data - linked data not created
